### Vote on predictions from different models

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
import cv2 as cv
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras import Model
from keras.utils.vis_utils import plot_model
import pathlib
import shutil

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, auc, classification_report, confusion_matrix, balanced_accuracy_score, roc_curve, roc_auc_score
import seaborn as sns

In [ ]:
data_dir = 'Train_Images'
val_dir = 'Val_Images'
test_dir = 'Test_Images'
#number of classes
num_classes = 2
img_height = 224
img_width = 224

Import Weights from different Models

In [ ]:
vgg19 = keras.models.load_model('Models/Vgg19_All_Limbs.h5')
resnet152_v2 = keras.models.load_model('Models/Resnet152_All_Limbs.h5')
mobilenet = keras.models.load_model('Models/Mobilenet_All_Limbs.h5')
inception_v3 = keras.models.load_model('Models/Inception_V3_All_Limbs.h5')
densenet = keras.models.load_model('Models/Densenet169_All_Limbs.h5')

Class Defination for General Class

In [ ]:
class_subset = sorted(os.listdir('Test_Images'), reverse=True)

### Data Generator as per the model

In [ ]:
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255
                                                            )
test_ds = test_gen.flow_from_directory(test_dir,
                                        target_size=(img_height, img_width),
                                        batch_size=1,
                                        class_mode='sparse',
                                        shuffle=False,
                                        classes=['Intact','Fractured'],
                                        seed=123)

#### Prediction for VGG19

In [ ]:
#true_classes = test_ds.classes
class_indices = {'Intact': 0, 'Fractured': 1}
class_indices = dict((v,k) for k,v in class_indices.items())
prediction_vgg19 = vgg19.predict(test_ds)
pred_classes_vgg19 = np.argmax(prediction_vgg19, axis=1) 
pred_classes_vgg19


#### Prediction for Resnet152

In [ ]:
#true_classes = test_ds.classes
class_indices = {'Intact': 0, 'Fractured': 1}
class_indices = dict((v,k) for k,v in class_indices.items())
prediction_resnet152 = resnet152_v2.predict(test_ds)
pred_classes_resnet152 = np.argmax(prediction_resnet152, axis=1) 
pred_classes_resnet152 

#### Prediction for Inception

In [ ]:
#true_classes = test_ds.classes
class_indices = {'Intact': 0, 'Fractured': 1}
class_indices = dict((v,k) for k,v in class_indices.items())
prediction_inception = inception_v3.predict(test_ds)
pred_classes_inception = np.argmax(prediction_inception, axis=1) 
pred_classes_inception 

#### For Densenet

In [ ]:
#true_classes = test_ds.classes
class_indices = {'Intact': 0, 'Fractured': 1}
class_indices = dict((v,k) for k,v in class_indices.items())
prediction_densenet = mobilenet.predict(test_ds)
pred_classes_densenet = np.argmax(prediction_densenet, axis=1) 
pred_classes_densenet

#### For MobileNet

In [ ]:
#true_classes = test_ds.classes
class_indices = {'Intact': 0, 'Fractured': 1}
class_indices = dict((v,k) for k,v in class_indices.items())
prediction_mobilenet = densenet.predict(test_ds)
pred_classes_mobilenet = np.argmax(prediction_mobilenet, axis=1) 
pred_classes_mobilenet

### Combined Prediction Array

In [ ]:
combined_preds = [pred_classes_vgg19, pred_classes_resnet152, pred_classes_inception, pred_classes_mobilenet, pred_classes_densenet]
voted_preds = np.apply_along_axis(lambda x: np.bincount(x).argmax(), 0, np.array(combined_preds))
voted_preds

### Metrices for the combined Predictionss

In [ ]:
# to get the true class values of the images, any datagenerator could be used
true_classes = test_ds.classes
true_classes

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, auc, classification_report, confusion_matrix, balanced_accuracy_score, roc_curve, roc_auc_score
import seaborn as sns

#calculating each of the metrices
accuracy = accuracy_score(true_classes, voted_preds)
balanced_accuracy = balanced_accuracy_score(true_classes, voted_preds)

precision = precision_score(true_classes, voted_preds)
f1 = f1_score(true_classes, voted_preds)

recall = recall_score(true_classes, voted_preds)

auc_mod = roc_auc_score(true_classes, voted_preds)

In [ ]:
print("Model Accuracy After Voting on Test Datset: {:.2f}%".format(accuracy * 100))
print("Balanced Accuracy After Voting on Test Datset: {:.2f}%".format(balanced_accuracy * 100))
print("Precision  After Voting on Test Datset: {:.2f}%".format(precision * 100))
print("Recall  After Voting on Test Datset: {:.2f}%".format(recall * 100))
print("F1-Score After Voting on Test Datset: {:.2f}%".format(f1 * 100))


#dictionary of the metrices
metrices = {'Accuracy' : [accuracy], 'Balanced_Accuracy': [balanced_accuracy], 'Precision': [precision], 'Recall': [recall], 'F1_Score': [f1], 'auc': auc_mod}

#create a dataframe of the metrices
df = pd.DataFrame.from_dict(metrices, orient='columns', dtype=None, columns=None)

#check for the file for saving metrices
file_save_path = 'Graphs/' + 'Voted_Result' + '/'
file_save_name = file_save_path + 'Voted_Result' + '_' + 'All_Limbs' +'_metrices.csv'

#check for the folder path, if exists skip else create the path
if not (os.path.exists(file_save_path)):
    os.makedirs(file_save_path)

df.to_csv(file_save_name, index=None)



Heatmap for Combined Predictions

In [ ]:
conf_mat = confusion_matrix(true_classes, voted_preds)
fig, ax = plt.subplots(figsize=(6,6))
ax = sns.heatmap(conf_mat, annot=True, cbar=False, square=True, fmt='d', cmap=plt.cm.Blues, xticklabels=class_subset, yticklabels=class_subset,annot_kws={'size': 18})
heatmap = ax.get_figure()

#check for the file for saving heatmap
figure_save_path = 'Graphs/' + 'Voted_Result' + '/'
figure_save_name = figure_save_path + 'Voted_Result' + '_' + 'All_Limbs' +'_combined_confusion_matrix.jpg'

#check for the folder path, if exists skip else create the path
if not (os.path.exists(figure_save_path)):
    os.makedirs(figure_save_path)

heatmap.savefig(figure_save_name, dpi=200)

#### Mismatched classification after combination

In [ ]:
combined_wrong = []

for i, file in enumerate(test_ds.filenames):
    if voted_preds[i]!=true_classes[i]:
        combined_wrong.append(file)
combined_wrong